# DATABASE PREPERATION

## data import

In [9]:
import pandas as pd
data = pd.read_csv('..\sales_data.csv')

In [10]:
data
data['Order_ID'] = data['Order_ID'] + 1

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
نکته: تنها جایی که 
NaN
داریم فقط توی 1 ستونه.
</p>

In [11]:
nan_counts = data.isna().sum()
empty_counts = (data == '').sum()

counts_df = pd.DataFrame({'NaN Count': nan_counts, 'Empty Count': empty_counts})

counts_df

,NaN Count,Empty Count
Order_ID,0,0
Branch,0,0
Order_Date,0,0
Order_Priority,0,0
Manufacturer,0,0
Model_Name,0,0
Category,0,0
Screen_Size,0,0
Screen,0,0
CPU,0,0


## Tables' creation

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
نکته: پسوورد دیتابیس خودتونو بزنید جای متغیر پسوورد و مطمئن باشید همه پکیج هارو توی 
venv
نصب کردید.
</p>

In [12]:
from urllib.parse import quote_plus
from sqlalchemy_utils import create_database, database_exists
import persiantools
from sqlalchemy import create_engine
from sqlalchemy import MetaData
import pymysql 
from sqlalchemy.orm import Session
from sqlalchemy import Table,Column,Integer, BIGINT, String, BigInteger, Boolean, Float, Unicode, Date,ForeignKey, Text 
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.mysql import LONGTEXT
meta = MetaData()

user = 'root'
password = 'hannibal1999' #your password here
host = 'localhost'
port = '3306'
db_name = 'lotech'


url = f'mysql://{user}:{password}@{host}:{port}/{db_name}?'
if not database_exists(url):
    create_database(url)


engine = create_engine(url , pool_pre_ping=True)
conn = engine.connect()
session = Session(bind=engine)

In [13]:
Branches = Table(
    'Branches', meta,
    Column('id' , Integer , primary_key=True),
    Column('branch_name' , String(50)),
    extend_existing=True,   
)
Priorities = Table(
    'Priorities', meta,
    Column('id' , Integer , primary_key=True),
    Column('priority' , String(5)),
    extend_existing=True,
)
meta.create_all(engine)
session.commit()

In [14]:
Orders = Table(
    'Orders', meta,
    Column('id' , BigInteger , primary_key=True),
    Column('shamsi_date' , String(15)),
    Column('date' , Date),
    Column('priority_id' , Integer , ForeignKey('Priorities.id')),
    Column('branch_id' , Integer , ForeignKey('Branches.id')),
    Column('ship_duration' , Integer),
    extend_existing=True,
)
RAM = Table(
    'RAM' , meta,
    Column('id' , Integer , primary_key=True),
    Column('capacity' , Integer),
    extend_existing=True,
)
Manufacturers = Table(
    'Manufacturers' , meta,
    Column('id' , Integer , primary_key=True),
    Column('Name' , String(100)),
    extend_existing=True,
)
OS = Table(
    'OS' , meta,
    Column('id' , Integer , primary_key=True),
    Column('type' , String(10)),
    Column('version' , String(5)),
    extend_existing=True,
)
Screen = Table(
    'Screen' , meta,
    Column('id' , Integer , primary_key=True),
    Column('size' , Float),
    Column('screen_type' , String(200)),
    extend_existing=True,
)
GPU = Table(
    'GPU', meta,
    Column('id' , Integer , primary_key=True),
    Column('mobile' , Boolean),
    Column('brand' , String(20)),
    Column('additional' , String(200)),
    extend_existing=True,
)
CPU = Table(
    'CPU', meta,
    Column('id' , Integer , primary_key=True),
    Column('Corei' , Integer),
    Column('speed' , Float),
    Column('brand' , String(20)),
    Column('additional' , String(200)),
    extend_existing=True,
)
meta.create_all(engine)
session.commit()

In [15]:

Category = Table(
    'Category', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    extend_existing=True
)

Products = Table(
    'Products', meta,
    Column('code' , BigInteger , primary_key=True),
    Column('model_name' , String(500)),
    Column('manufacturer_id' , Integer , ForeignKey('Manufacturers.id') ),
    Column('category_id' , Integer, ForeignKey('Category.id')),
    Column('screen_id' , Integer , ForeignKey('Screen.id')),
    Column('CPU_id' , Integer , ForeignKey('CPU.id')),
    Column('RAM_id' , Integer , ForeignKey('RAM.id') ),
    Column('storage' , String(200)),
    Column('GPU_id' , Integer , ForeignKey('GPU.id')),
    Column('OS_id' , Integer , ForeignKey('OS.id')),
    Column('weight' , Float),
    Column('price' , BigInteger),
    extend_existing=True,
)
meta.create_all(engine)
session.commit()


In [16]:
Order_Details = Table(
    'Order_Details' , meta,
    Column('id' , BigInteger , primary_key=True),
    Column('order_id', BigInteger , ForeignKey('Orders.id') ),
    Column('product_code' , BigInteger , ForeignKey('Products.code') ),
    Column('quantity' , Integer),
    Column('price' , BigInteger),
    Column('discount' , Integer),
    Column('total_price' , BigInteger),
    Column('profit' , Integer),
    extend_existing=True,
)
meta.create_all(engine)
session.commit()

## Data Cleaning and passing into the database

## نکات مهم:

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
برای پاس دادن دیتا به تیبل‌های دیتابیس حواستون به اسم و تایپ ستون‌هاباشه
.
to_sql
به اسم و تایپا دقت میکنه و  ترتیب ستون ها در دیتافریم مهم نیست
.

لینک اسکیما برای راحتی کار:
https://drawsql.app/teams/quera-2/diagrams/lotech
</p>

### Mahdi TO-DO 1


In [17]:
branches_df = pd.DataFrame(data['Branch'])
branches_df.rename(columns={'Branch': 'branch_name'}, inplace=True)
branches_df.drop_duplicates(inplace=True, ignore_index=True)
branches_df.index = [i for i in range(1, len(branches_df) + 1)]
branches_df.reset_index(inplace=True)
branches_df.rename(columns={'index': 'id','Branch': 'branch_name'}, inplace=True)

branches_df.to_sql('branches', con=engine, if_exists='append', index=False)
meta.create_all(engine)
session.commit()
branches_df
# columns needed: branch_name , id

,id,branch_name
0,1,Hamedan
1,2,Kermanshah
2,3,Rasht
3,4,Karaj
4,5,Mashhad
5,6,Yazd
6,7,Ahvaz
7,8,Arak
8,9,Isfahan
9,10,Tehran


### Parinaz TO-DO 1 

In [18]:
data

,Order_ID,Branch,Order_Date,Order_Priority,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,...,GPU,OS,OS_Version,Weight,Price,Quantity,Discount,Total_Price,Profit,Ship_Duration
0,1,Hamedan,1396-10-26,H,Dell,Vostro 3568,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,...,AMD Radeon R5 M420,Windows,10,2.18kg,3450000,1,0,3450000,250000,3
1,2,Kermanshah,1394-04-21,H,Lenovo,Yoga 720-13IKB,2 in 1 Convertible,"13.3""",IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,...,Intel HD Graphics 620,Windows,10,1.3kg,3400000,1,0,3400000,380000,2
2,3,Rasht,1402-05-24,M,Lenovo,IdeaPad Y700-15ISK,Notebook,"15.6""",IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,...,Nvidia GeForce GTX 960M,Windows,10,2.6kg,44900000,1,0,44900000,5630000,5
3,4,Kermanshah,1397-06-17,H,Dell,Inspiron 5370,Ultrabook,"13.3""",IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,...,AMD Radeon 530,Windows,10,1.4kg,12550000,1,0,12550000,1130000,3
4,5,Karaj,1392-09-30,L,Acer,Aspire E5-576G,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,...,Nvidia GeForce 940MX,Windows,10,2.23kg,2050000,1,0,2050000,180000,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017076,1017077,Kermanshah,1395-11-06,M,Lenovo,Legion Y520-15IKBN,Gaming,"15.6""",IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,...,Nvidia GeForce GTX 1050,Windows,10,2.4kg,3800000,1,0,3800000,460000,4
1017077,1017078,Ahvaz,1391-04-24,M,Lenovo,V330-15IKB (i5-8250U/4GB/500GB/FHD/W10),Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,...,Intel HD Graphics 620,Windows,10,2.05kg,1300000,1,0,1300000,120000,6
1017078,1017079,Tabriz,1399-05-18,H,Dell,Vostro 5568,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,...,Intel HD Graphics 620,Linux,NaN,1.98kg,16700000,1,0,16700000,1700000,2
1017079,1017080,Mashhad,1394-02-04,L,HP,Envy 13-AB020nr,Ultrabook,"13.3""",IPS Panel Quad HD+ 3200x1800,Intel Core i7 7500U 2.7GHz,...,Intel HD Graphics 620,Windows,10,1.34kg,3750000,1,0,3750000,420000,10


In [19]:
#  Parinaz TO-DO 
prio = ['L' , 'M' , 'H' , 'C']
prio = pd.DataFrame(prio)
prio = prio.assign(id=range(1, len(prio) + 1))
prio.rename(columns={0 : 'priority'} , inplace=True)
Priorities = Table(
    'Priorities', meta,
    Column('id' , Integer, primary_key=True),
    Column('priority' , String(5)),
    extend_existing=True,
)
prio.to_sql('Priorities' , con=engine ,  if_exists='append' , index=False)
meta.create_all(engine)
session.commit()



priorities_encoding = {
    "C" : 4,
    "H" : 3,
    "M" : 2 ,
    "L" : 1,
}
orders_df = data[['Order_ID' , 'Order_Date' , 'Order_Priority' , 'Branch' , 'Ship_Duration']]

orders_df['shamsi_date'] = orders_df['Order_Date']

orders_df['Order_Priority'].replace(priorities_encoding, inplace=True)
orders_df = orders_df.merge(branches_df, left_on='Branch', right_on='branch_name', how='left')
# Drop the original 'Branch' column and rename the 'id' column to 'Branch'
orders_df.drop(columns=['Branch'], inplace=True)
orders_df.drop(columns=['branch_name'], inplace=True)
orders_df.rename(columns={'id': 'branch_id' , 'Order_ID':'id' , 'Order_Date':'date' , 'Ship_Duration' : 'ship_duration' , 'Order_Priority' : 'priority_id'}, inplace=True)



C:\Users\Parinaz\AppData\Local\Temp\ipykernel_15628\2290918695.py:12: UserWarning: The provided table name 'Priorities' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  prio.to_sql('Priorities' , con=engine ,  if_exists='append' , index=False)
C:\Users\Parinaz\AppData\Local\Temp\ipykernel_15628\2290918695.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['shamsi_date'] = orders_df['Order_Date']
C:\Users\Parinaz\AppData\Local\Temp\ipykernel_15628\2290918695.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [20]:
orders_df

,id,date,priority_id,ship_duration,shamsi_date,branch_id
0,1,1396-10-26,3,3,1396-10-26,1
1,2,1394-04-21,3,2,1394-04-21,2
2,3,1402-05-24,2,5,1402-05-24,3
3,4,1397-06-17,3,3,1397-06-17,2
4,5,1392-09-30,1,8,1392-09-30,4
...,...,...,...,...,...,...
1017076,1017077,1395-11-06,2,4,1395-11-06,2
1017077,1017078,1391-04-24,2,6,1391-04-24,7
1017078,1017079,1399-05-18,3,2,1399-05-18,15
1017079,1017080,1394-02-04,1,10,1394-02-04,5


In [46]:
from persiantools.jdatetime import JalaliDate
import jalali_pandas
df = orders_df
def convert_to_gregorian(shamsi_date):
    shamsi_parts = shamsi_date.split('-')
    shamsi_year = int(shamsi_parts[0])
    shamsi_month = int(shamsi_parts[1])
    shamsi_day = int(shamsi_parts[2])

    # Check if the day value is valid for the given month
    if shamsi_day == 30 and shamsi_month in [12, 11, 10, 9, 8 , 7 , ]:
        shamsi_day = 29
    elif shamsi_day > 30:
        shamsi_day = 29
    if shamsi_year >= 1402:
        gregorian_date = JalaliDate(1402, shamsi_month, shamsi_day).to_gregorian()
    elif shamsi_year == 1401:
        gregorian_date = JalaliDate(1401, shamsi_month, shamsi_day).to_gregorian()
    else:
        gregorian_date = JalaliDate(shamsi_year, shamsi_month, shamsi_day).to_gregorian()
    gregorian_date_str = gregorian_date.strftime('%Y-%m-%d')

    return gregorian_date_str

df['date']=df['date'].apply(convert_to_gregorian)
orders_df = df



In [48]:
orders_df['date'].sort_values(ascending=False)



579194    2024-03-19
546511    2024-03-19
441183    2024-03-19
708734    2024-03-19
917686    2024-03-19
             ...    
154619    2023-03-21
295146    2023-03-21
614583    2023-03-21
571528    2023-03-21
280445    2023-03-21
Name: date, Length: 1017081, dtype: object

In [49]:
x = orders_df['shamsi_date']
x[436903]

'1402-07-19'

In [50]:
orders_df['id'].drop_duplicates(inplace=True)
orders_df.to_sql('Orders' , con=engine ,  if_exists='append' , index=False)

C:\Users\Parinaz\AppData\Local\Temp\ipykernel_15628\2858455758.py:2: UserWarning: The provided table name 'Orders' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  orders_df.to_sql('Orders' , con=engine ,  if_exists='append' , index=False)


1017081

### Mahdi TO-DO 2

In [51]:
## MAHDi TO-DO
screen_df = pd.DataFrame(data[['Screen_Size', 'Screen']])
screen_df.drop_duplicates(subset=['Screen_Size', 'Screen'], inplace=True, ignore_index=True)
screen_df.index = [*range(1, len(screen_df) + 1)]
screen_df.reset_index(inplace=True)
screen_df_copy = screen_df.copy()

screen_df.rename(columns={'index':'id', 'Screen_Size':'size', 'Screen':'screen_type'}, inplace=True)
screen_df["size"] = screen_df['size'].apply(lambda x: float(x.replace('"', '')))
screen_df.to_sql('screen', con=engine, index=False, if_exists='append')
meta.create_all(engine)
session.commit()
#screen_df_copy

In [52]:
os_df = pd.DataFrame(data[['OS', 'OS_Version']])
os_df.drop_duplicates(subset=['OS', 'OS_Version'], inplace=True)
os_df.index = [*range(1, len(os_df) + 1)]
os_df.reset_index(inplace=True)
os_df.rename(columns={'index': 'id', 'OS':'type', 'OS_Version':'version'}, inplace=True)

os_df.to_sql('os', con=engine, index=False, if_exists='append')
meta.create_all(engine)
session.commit()
#os_df

In [53]:
ram_df = pd.DataFrame(data['RAM'])
ram_df.drop_duplicates(inplace=True)
ram_df.index = [*range(1, len(ram_df) + 1)]
ram_df.reset_index(inplace=True)
ram_df_copy = ram_df.copy()
ram_df['RAM'] = ram_df['RAM'].apply(lambda x : x.replace('GB', ''))



ram_df.rename(columns={'index': 'id', 'RAM':'capacity'}, inplace=True)
ram_df.to_sql('ram', con=engine, if_exists='append', index=False)
meta.create_all(engine)
session.commit()
#ram_df_copy

In [54]:
manufacturers_df = pd.DataFrame(data['Manufacturer'])
manufacturers_df.drop_duplicates(inplace=True)
manufacturers_df.index = [*range(1, len(manufacturers_df) + 1)]
manufacturers_df.reset_index(inplace=True)
manufacturers_df.rename(columns={'index':'id', 'Manufacturer':'name'}, inplace=True)

manufacturers_df.to_sql('manufacturers', con=engine, if_exists='append', index=False)
meta.create_all(engine)
session.commit()
#manufacturers_df

In [55]:
#Storage 

category_df = pd.DataFrame(data['Category'])
category_df.drop_duplicates(inplace=True, ignore_index=True)
category_df.index = [*range(1, len(category_df) + 1)]
category_df.reset_index(inplace=True)
category_df.rename(columns={'index':'id', 'Category': 'name'}, inplace=True)

category_df.to_sql('category', con=engine, if_exists='append', index=False)
meta.create_all(engine)
session.commit()
#category_df

In [56]:
products_df = data[['Model_Name', 'Manufacturer', 'Category', 'Screen_Size', 'Screen', 'CPU', 'RAM', 'Storage', 'GPU', 'OS', 'OS_Version', 'Weight', 'Price']].copy()
products_df = products_df.merge(category_df, left_on='Category', right_on='name', how='left')
products_df.drop(columns=['name', 'Category'], axis=1, inplace=True)
products_df.rename(columns={'id':'category_id'}, inplace=True)
#products_df

In [57]:
products_df = products_df.merge(manufacturers_df, left_on='Manufacturer', right_on='name', how='left')
products_df.drop(columns=['name', 'Manufacturer'], axis=1, inplace=True)
products_df.rename(columns={'id':'manufacturer_id'}, inplace=True)
#products_df

In [58]:
products_df = products_df.merge(os_df, left_on=['OS', 'OS_Version'], right_on=['type', 'version'], how='left')
products_df.drop(columns=['type', 'version', 'OS', 'OS_Version'], axis=1, inplace=True)
products_df.rename(columns={'id':'os_id'}, inplace=True)
#products_df

In [59]:
products_df = products_df.merge(ram_df_copy, left_on='RAM', right_on='RAM', how='left')
products_df.drop(columns=['RAM'], axis=1, inplace=True)
products_df.rename(columns={'index':'ram_id'}, inplace=True)
#products_df

In [60]:
products_df = products_df.merge(screen_df_copy, left_on=['Screen', 'Screen_Size'], right_on=['Screen', 'Screen_Size'], how='left')
products_df.drop(columns=['Screen', 'Screen_Size'], axis=1, inplace=True)
products_df.rename(columns={'index':'screen_id'}, inplace=True)
#products_df

## PARINAZ TO-DO 2

In [61]:
## PARINAZ TO-DO 
cpu_df = pd.DataFrame(data['CPU']).drop_duplicates()


cpu_df_copy = cpu_df.copy()
cpu_df_copy.index = [*range(1, len(cpu_df)+1)]
cpu_df_copy.reset_index(inplace=True)
cpu_df['brand'] = cpu_df['CPU'].str.extract(r'^(\w+)')
cpu_df['speed'] = cpu_df['CPU'].str.extract(r'(\d+\.\d+)GHz$')
cpu_df['Corei'] = cpu_df['CPU'].str.extract(r'Core i(\d+)')
cpu_df.rename(columns={'CPU' : 'additional' } , inplace=True)
cpu_df = cpu_df.assign(id=range(1, len(cpu_df) + 1))
cpu_df['Corei'].fillna(0 , inplace=True)
cpu_df = cpu_df.astype({'speed': 'float64'})
cpu_df = cpu_df.astype({'Corei': 'int64'})
cpu_df = cpu_df.astype({'additional': 'str'})
cpu_df = cpu_df.astype({'brand': 'str'})

cpu_df.to_sql('CPU' , con=engine ,  if_exists='append' , index=False)
cpu_df_copy

C:\Users\Parinaz\AppData\Local\Temp\ipykernel_15628\369764568.py:19: UserWarning: The provided table name 'CPU' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  cpu_df.to_sql('CPU' , con=engine ,  if_exists='append' , index=False)


,index,CPU
0,1,Intel Core i5 7200U 2.5GHz
1,2,Intel Core i7 6700HQ 2.6GHz
2,3,Intel Core i7 8550U 1.8GHz
3,4,AMD FX 8800P 2.1GHz
4,5,Intel Core i7 7500U 2.7GHz
...,...,...
113,114,AMD A4-Series 7210 2.2GHz
114,115,Intel Atom Z8350 1.92GHz
115,116,AMD E-Series 9000 2.2GHz
116,117,Intel Xeon E3-1535M v6 3.1GHz


In [62]:
gpu_df = pd.DataFrame(data['GPU'].drop_duplicates())

gpu_df_copy = gpu_df.copy()
gpu_df_copy.index = [*range(1, len(gpu_df)+1)]
gpu_df_copy.reset_index(inplace=True)
gpu_df['brand'] = gpu_df['GPU'].str.extract(r'^(\w+)')
gpu_df['mobile'] = gpu_df['GPU'].str.contains('M')
'''
gpu_df['mobile'] = gpu_df['mobile'].astype(str)'''
gpu_df.rename(columns={'GPU' : 'additional' } , inplace=True)
gpu_df = gpu_df.assign(id=range(1, len(gpu_df) + 1))
gpu_df = gpu_df.astype({'mobile': 'bool'})
gpu_df = gpu_df.astype({'additional': 'str'})
gpu_df = gpu_df.astype({'brand': 'str'})

gpu_df.to_sql('GPU' , con=engine ,  if_exists='append' , index=False)
gpu_df_copy

C:\Users\Parinaz\AppData\Local\Temp\ipykernel_15628\3565408835.py:16: UserWarning: The provided table name 'GPU' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  gpu_df.to_sql('GPU' , con=engine ,  if_exists='append' , index=False)


,index,GPU
0,1,AMD Radeon R5 M420
1,2,Intel HD Graphics 620
2,3,Nvidia GeForce GTX 960M
3,4,AMD Radeon 530
4,5,Nvidia GeForce 940MX
...,...,...
105,106,Nvidia GTX 980 SLI
106,107,Intel Iris Graphics 550
107,108,AMD Radeon R3
108,109,Nvidia GeForce GTX1080


## MAHDI TO_DO 2

In [63]:
products_df = products_df.merge(cpu_df_copy, left_on='CPU', right_on='CPU', how='left')
products_df.drop(columns=['CPU'], axis=1, inplace=True)
products_df.rename(columns={'index':'cpu_id'}, inplace=True)
products_df

,Model_Name,Storage,GPU,Weight,Price,category_id,manufacturer_id,os_id,ram_id,screen_id,cpu_id
0,Vostro 3568,256GB SSD,AMD Radeon R5 M420,2.18kg,3450000,1,1,1,1,1,1
1,Yoga 720-13IKB,256GB SSD,Intel HD Graphics 620,1.3kg,3400000,2,2,1,1,2,1
2,IdeaPad Y700-15ISK,1TB HDD,Nvidia GeForce GTX 960M,2.6kg,44900000,1,2,1,1,3,2
3,Inspiron 5370,256GB SSD,AMD Radeon 530,1.4kg,12550000,3,1,1,1,4,3
4,Aspire E5-576G,256GB SSD,Nvidia GeForce 940MX,2.23kg,2050000,1,3,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1017076,Legion Y520-15IKBN,1TB HDD,Nvidia GeForce GTX 1050,2.4kg,3800000,4,2,1,2,3,7
1017077,V330-15IKB (i5-8250U/4GB/500GB/FHD/W10),500GB HDD,Intel HD Graphics 620,2.05kg,1300000,1,2,1,2,1,14
1017078,Vostro 5568,1TB HDD,Intel HD Graphics 620,1.98kg,16700000,1,1,3,1,1,1
1017079,Envy 13-AB020nr,256GB SSD,Intel HD Graphics 620,1.34kg,3750000,3,4,1,1,27,5


In [64]:
products_df = products_df.merge(gpu_df_copy, left_on='GPU', right_on='GPU', how='left')
products_df.drop(columns=['GPU'], axis=1, inplace=True)
products_df.rename(columns={'index':'gpu_id'}, inplace=True)
products_df

,Model_Name,Storage,Weight,Price,category_id,manufacturer_id,os_id,ram_id,screen_id,cpu_id,gpu_id
0,Vostro 3568,256GB SSD,2.18kg,3450000,1,1,1,1,1,1,1
1,Yoga 720-13IKB,256GB SSD,1.3kg,3400000,2,2,1,1,2,1,2
2,IdeaPad Y700-15ISK,1TB HDD,2.6kg,44900000,1,2,1,1,3,2,3
3,Inspiron 5370,256GB SSD,1.4kg,12550000,3,1,1,1,4,3,4
4,Aspire E5-576G,256GB SSD,2.23kg,2050000,1,3,1,2,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
1017076,Legion Y520-15IKBN,1TB HDD,2.4kg,3800000,4,2,1,2,3,7,9
1017077,V330-15IKB (i5-8250U/4GB/500GB/FHD/W10),500GB HDD,2.05kg,1300000,1,2,1,2,1,14,2
1017078,Vostro 5568,1TB HDD,1.98kg,16700000,1,1,3,1,1,1,2
1017079,Envy 13-AB020nr,256GB SSD,1.34kg,3750000,3,4,1,1,27,5,2


In [65]:
products_df.drop_duplicates(subset=['Model_Name', 'manufacturer_id'], inplace=True, ignore_index=True)
products_df.index = [*range(1, len(products_df) + 1)]
products_df.reset_index(inplace=True)
products_df.rename(columns={
    'index':'code',
    'category':'category_id',
    'Manufacturer':'manufacturer_id'

}, inplace=True)

products_df['Weight'] = products_df['Weight'].apply(lambda x: x.replace('kg', ''))
products_df['Weight'] = products_df['Weight'].apply(lambda x: float(x.replace('s', '')))
products_df.to_sql('products', if_exists='append', index=False, con=engine)
meta.create_all(engine)
session.commit()
products_df

,code,Model_Name,Storage,Weight,Price,category_id,manufacturer_id,os_id,ram_id,screen_id,cpu_id,gpu_id
0,1,Vostro 3568,256GB SSD,2.18,3450000,1,1,1,1,1,1,1
1,2,Yoga 720-13IKB,256GB SSD,1.30,3400000,2,2,1,1,2,1,2
2,3,IdeaPad Y700-15ISK,1TB HDD,2.60,44900000,1,2,1,1,3,2,3
3,4,Inspiron 5370,256GB SSD,1.40,12550000,3,1,1,1,4,3,4
4,5,Aspire E5-576G,256GB SSD,2.23,2050000,1,3,1,2,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
615,616,V110-15IAP (N3350/4GB/128GB/No,128GB SSD,1.90,800000,1,2,6,2,6,9,12
616,617,V131 (X5-Z8350/4GB/32GB/FHD/W10),32GB Flash Storage,1.35,750000,1,19,1,2,11,110,42
617,618,X541NA (N3350/4GB/1TB/FHD/W10),1TB HDD,2.00,1200000,1,6,1,2,1,9,12
618,619,GT73EVR 7RE,512GB SSD + 1TB HDD,4.14,9550000,4,7,1,3,9,7,64


In [66]:
products_df.nunique()

code               620
Model_Name         618
Storage             30
Weight             152
Price              347
category_id          6
manufacturer_id     19
os_id                9
ram_id               9
screen_id           67
cpu_id              82
gpu_id              86
dtype: int64

In [67]:
list(products_df.columns)

['code',
 'Model_Name',
 'Storage',
 'Weight',
 'Price',
 'category_id',
 'manufacturer_id',
 'os_id',
 'ram_id',
 'screen_id',
 'cpu_id',
 'gpu_id']

In [68]:
orders_df

,id,date,priority_id,ship_duration,shamsi_date,branch_id
0,1,2023-09-16,3,3,1396-10-26,1
1,2,2024-03-14,3,2,1394-04-21,2
2,3,2023-04-14,2,5,1402-05-24,3
3,4,2023-05-07,3,3,1397-06-17,2
4,5,2023-08-20,1,8,1392-09-30,4
...,...,...,...,...,...,...
1017076,1017077,2023-09-27,2,4,1395-11-06,2
1017077,1017078,2024-03-16,2,6,1391-04-24,7
1017078,1017079,2023-04-07,3,2,1399-05-18,15
1017079,1017080,2023-12-27,1,10,1394-02-04,5


In [69]:
order_details_df = data[['Order_ID', 'Model_Name', 'Quantity', 'Price', 'Discount', 'Total_Price', 'Profit']].copy()
order_details_df = order_details_df.merge(products_df, left_on='Model_Name', right_on='Model_Name', how='left')
order_details_df.drop_duplicates(inplace=True, ignore_index=True)
list(products_df.columns)
list(products_df.columns)
order_details_df.drop(columns=['Model_Name','Storage',
 'Weight',
 'Price_y',
 'category_id',
 'manufacturer_id',
 'os_id',
 'ram_id',
 'screen_id',
 'cpu_id',
 'gpu_id'], inplace=True)
order_details_df.index = [*range(1, len(order_details_df) + 1)]
order_details_df.reset_index(inplace=True)
order_details_df.rename(columns={
    'index':'id',
    'code':'product_code',
    'Price_x':'price'
}, inplace=True)
order_details_df
order_details_df['Total_Price'] = order_details_df['Total_Price'].apply(lambda x: int(x))

order_details_df


,id,Order_ID,Quantity,price,Discount,Total_Price,Profit,product_code
0,1,1,1,3450000,0,3450000,250000,1
1,2,2,1,3400000,0,3400000,380000,2
2,3,3,1,44900000,0,44900000,5630000,3
3,4,4,1,12550000,0,12550000,1130000,4
4,5,5,1,2050000,0,2050000,180000,5
...,...,...,...,...,...,...,...,...
1018200,1018201,1017077,1,3800000,0,3800000,460000,10
1018201,1018202,1017078,1,1300000,0,1300000,120000,394
1018202,1018203,1017079,1,16700000,0,16700000,1700000,15
1018203,1018204,1017080,1,3750000,0,3750000,420000,532


In [70]:
order_details_df.to_sql('order_details', con=engine, if_exists='append', index=False)
meta.create_all(engine)
session.commit()
